In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

## Decision Tree Leaner

In [14]:
class DTlearner(object):
   
#   class for tree building
    class Node:
        def __init__(self,data=None):
            self.data=data
            self.left=None
            self.right=None
        
        
    
    def __init__(self, criterion='squared_error',size=1):
        self._root=None
        
        # Error criteria
        self._criterion = criterion
        
        # Batch size jispr splitting hona bnd hojae
        self._SIZE=size
        
        self.count=0
        
        
    def add_evidence(self,X,y):
        self._decision_tree(X,y)
    
    def _decision_tree(self,X,y):
        
        self._root = self._build_tree(X,y,self._root)
        
    def _build_tree(self,X,y,node):
        # Selecting best feature and its value
        feature, value = self._get_best_feature(X,y)
        
        
        # Making a node of that features to make this feature as a Root Node
        node = self.Node([feature,value])
        
        # Concat the dataframe 
        concat_df = pd.concat([X,y],axis=1)
        
        # creating dataframes for left and right split on the basis of best feature and its values
        # split less than best value goes on left side and greater one goes on right side
        left_split = concat_df[concat_df[feature]<value]
        right_split = concat_df[concat_df[feature]>value]
        
        
        # When both data splits are greater than our batch size 
        if (left_split.shape[0]>self._SIZE) & (right_split.shape[0]>self._SIZE):
            # send the left split on the left side of the node
            
            node.left = self._build_tree(left_split.iloc[:,:-1],left_split.iloc[:,-1:],node.left)
            
            # send the right split on the right side of the node
            node.right = self._build_tree(right_split.iloc[:,:-1],right_split.iloc[:,-1:],node.right)
            
        # When both are less than batch size
        elif (left_split.shape[0]<=self._SIZE) & (right_split.shape[0]<=self._SIZE):

            # There are 3 possibilities
            
#             1) when both are != 0 then simply take the mean of Y attribute and make it as leaf_node
            if (left_split.shape[0]!=0) & (right_split.shape[0]!=0):

                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])
           
        
#           2) when left_split is 0 and right_split>0 then make only a node on right side 
            elif (left_split.shape[0]==0):
              
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])

#           3) when right_split is 0 and left>0 then make only a node on left side 
                
            elif (right_split.shape[0]==0):
                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
        
        
#         When only left is small but not the right one
        elif (left_split.shape[0]<=self._SIZE):
            
#             Make a leaf_node on left side and expand the tree on the right side
            if (left_split.shape[0]!=0):
                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
            
            node.right = self._build_tree(right_split.iloc[:,:-1],right_split.iloc[:,-1:],node.right)

#       When only right is small but not the left
        elif (right_split.shape[0]<=self._SIZE):
            
            if (right_split.shape[0]!=0):
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])
                
            node.left = self._build_tree(left_split.iloc[:,:-1],left_split.iloc[:,-1:],node.left)



        return node
        
    def _get_best_feature(self,X,y):
        points= {}
      


    #Selecting all features

        for feature in X.columns:
            i=0

#             Sort the values in ascending order
            _arr= X[feature].sort_values(ascending=True).values
            errors = []
            averages = []
            
#             Calculate best value for the selected feature having min error value
            while i<X.shape[0]-1:
                avg = sum(_arr[i:i+2])/2
            
                left_mean = np.mean(_arr[_arr<avg])
                right_mean = np.mean(_arr[_arr>avg])


                if self._criterion == 'squared_error':
                    error = sum((np.array(y)-left_mean)**2) + sum((np.array(y)-right_mean)**2)
             
                elif self._criterion == 'mse':
                    error = (sum((np.array(y)-left_mean)**2) + sum((np.array(y)-right_mean)**2))/len(y)
                
                elif self._criterion == 'mae':
                    error = (sum(abs(np.array(y)-left_mean)) + sum(abs(np.array(y)-right_mean)))/len(y)

                errors.append(error)
                averages.append(avg)
                
                i=i+1
                
#             Dictionay me ek feature ke tmam errors me se sbse min error wali avg value jae gi us feature ke against
            points[feature] = [averages[np.argmin(errors)],min(errors)]
             
        
        feature = ''
        ssr = np.inf
        point = 0
#       upr ka process hr ek feature pr chalega tmam features ke against unki avg value or min errors agai hain
#       ab hm yhan woh ek feature dekhain gai jiska error sbse km hoga and then uski value ko return kren gai
        for key, value in points.items():
            if ssr >= value[1]:
                feature = key
                ssr = value[1]
                point = value[0]

        
        return (feature,point)
    
    # It take each row and start predicting values by passing each row in _predict function
    # ravel : will convert the predicted array into 1D array
    def query(self,X_test):
        results = []
        
        for index,row in X_test.iterrows():
            results.append(self._predict(row,self._root))
            
        return np.ravel(results)
    
    
    def _predict(self,row,node):
        """Params: 
            row: A entry of data
            node: the node of the tree"""
        
        if node is None:
            return 'None return'
        
        # If Node type is leaf_node then return the value part of that node
        elif node.data[0]=='leaf_node':
            return node.data[1].values
        
        # Select the feature value of that feature which is specified in node feature part
        value = row[node.data[0]]
        # If node value is greater then move the tree to the left side
        if node.data[1]>value:
            # r stores the predicted value
            r = self._predict(row,node.left)
            
            # Here None return means that at the left side there is no node hence we have to move the tree to the right so that we can get the prediction
            if r=='None return':
                return self._predict(row,node.right)
            return r
        else:
            # If node value is less then move the tree to the left side
            
            r = self._predict(row,node.right)
            
            # Here None return means that at the right side there is no node hence we have to move the tree to the left so that we can get the prediction
            if r=='None return':
                return self._predict(row,node.left)
            
            return r
        
    
    # This function will print the whole tree in Left-ROOT-Right format
    def inorder(self,node):
        
        if node is None:
            
            return
        
        self.inorder(node.left)
        print(node.data[0],node.data[1])
        self.inorder(node.right)
    
    # This function will give us the main root of the tree
    def getRoot(self):
        return self._root
    
    

## Regression Tree Leaner

In [15]:
class RTlearner(object):
   
#   class for tree building
    class Node:
        def __init__(self,data=None):
            self.data=data
            self.left=None
            self.right=None
        
        
    
    def __init__(self,size=20):
        self._root=None
        self._SIZE=size
        
    def add_evidence(self,X,y):
        self._decision_tree(X,y)
    
    def _decision_tree(self,X,y):
        
        self._root = self._build_tree(X,y,self._root)
    
    # this function builds the tree same as i mention in above learner
    def _build_tree(self,X,y,node):
        feature, value = self._get_best_feature(X)
        
        
        node = self.Node([feature,value])
        
        concat_df = pd.concat([X,y],axis=1)
        
        left_split = concat_df[concat_df[feature]<value]
        right_split = concat_df[concat_df[feature]>value]
        
        
        if (left_split.shape[0]>self._SIZE) & (right_split.shape[0]>self._SIZE):
            
            node.left = self._build_tree(left_split.iloc[:,:-1],left_split.iloc[:,-1:],node.left)
            
            node.right = self._build_tree(right_split.iloc[:,:-1],right_split.iloc[:,-1:],node.right)
            
        elif (left_split.shape[0]<=self._SIZE) & (right_split.shape[0]<=self._SIZE):

            if (left_split.shape[0]!=0) & (right_split.shape[0]!=0):
                
                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])
           
            
            elif (left_split.shape[0]==0) & (right_split.shape[0]==0):
                    return
            elif (left_split.shape[0]==0):
                
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])

                
            elif (right_split.shape[0]==0):
                
                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
        
        
        elif (left_split.shape[0]<=self._SIZE):
            
            if (left_split.shape[0]!=0):
                node.left = self.Node(['leaf_node',np.mean(left_split.iloc[:,-1:])])
            
            node.right = self._build_tree(right_split.iloc[:,:-1],right_split.iloc[:,-1:],node.right)

        elif (right_split.shape[0]<=self._SIZE):
            
            if (right_split.shape[0]!=0):
                node.right = self.Node(['leaf_node',np.mean(right_split.iloc[:,-1:])])
                
            node.left = self._build_tree(left_split.iloc[:,:-1],left_split.iloc[:,-1:],node.left)



        return node
        
    # here a random feature is selected for the tree building
    # we select 2 random values of that feature and calulate the mean and return them
    def _get_best_feature(self,X):
        f_ind=np.random.randint(X.shape[1])
        feature = X.columns[f_ind]

        i = np.random.randint(X.shape[0])
        j = np.random.randint(X.shape[0])

        value = (X[feature].iloc[i] + X[feature].iloc[j])/2
        return (feature,value)

        
    
    def query(self,X_test):
        results = []
        
        for index,row in X_test.iterrows():
            results.append(self._predict(row,self._root))
            
        return np.ravel(results)
    
    def _predict(self,row,node):
        
        if node is None:
            return 'None return'
        
        elif node.data[0]=='leaf_node':
            return node.data[1].values
        
        
        value = row[node.data[0]]
        if node.data[1]>value:
            r = self._predict(row,node.left)
            
            if r=='None return':
                return self._predict(row,node.right)
            return r
        else:
            
            r = self._predict(row,node.right)
            
            if r=='None return':
                return self._predict(row,node.left)
            
            return r
        
    
    def inorder(self,node):
        
        if node is None:
            
            return
        
        self.inorder(node.left)
        print(node.data[0],node.data[1])
        self.inorder(node.right)
    
    def getRoot(self):
        return self._root
    
    


## Bag Learner

In [16]:
class BagLearner:
    
    """BagLearner train multiple learner or bags of leraner to predict the values"""
    def __init__(self,learner,kwargs={'size':1}, bags=10, boost= False, verbose=False):
        self.kwargs = kwargs
        self.learner = learner
        self.boost = boost
        self.verbose = verbose
        self.bags = bags
        self.learners = []
        
        if verbose:
            print('BagLeaner()')
        
        
        
    def add_evidence(self,X_train,y_train):
        
        # if boost is true then we use bagging technique and then train the individual model
        if self.boost:
            df = pd.concat([X_train,y_train],axis=1)
           
            for i in range(0,self.bags):
                df2 = df.sample(df.shape[0],replace=True)
            
                self.learners.append(self.learner(**self.kwargs))
                self.learners[i].add_evidence(X_train,y_train)
        else:
            
             for i in range(0,self.bags):
                self.learners.append(self.learner(**self.kwargs))
                self.learners[i].add_evidence(X_train,y_train)
    
    # Here every gives us multiple prediction for a particular entry we take the mean of all prediction
    def query(self,X_test):
        y_pred = []
        
        for i in range(len(self.learners)):
            y_pred.append(self.learners[i].query(X_test))
        
        return np.mean(y_pred,axis=0)

## Insane Learner

In [21]:
class InsaneLearner(object):
    """It Implements 20 BagLearners"""
    def __init__(self,verbose=False):
        
        self.learners =[]
        if verbose:
            print('InsaneLearner()')
            
        for i in range(20):
            self.learners.append(BagLearner(DTlearner,bags=20,verbose= verbose))
            
    def add_evidence(self,X_train,y_train):
        for i in range(20):
            self.learners[i].add_evidence(X_train,y_train)
            
    def query(self,X_test):
        
        y_pred  = []
        
        for i in range(20):
            y_pred.append(self.learners[i].query(X_test))
            
        return np.mean(y_pred,axis=0)

## Linear Regression Learner

In [22]:
class LinRegLearner(object):
    
    def __init__(self):
        pass
   
    # Generating an array of ones greater than the shape of X_train by introducing one extra column
    # Replace the X values with X_train
    def add_evidence(self,X_train,y_train):
        X = np.ones([X_train.shape[0],X_train.shape[1]+1])
        X[:,0:X_train.shape[1]] = X_train
        
        # Calculate the coef of regression
        self.coef = np.linalg.lstsq(X, y_train,rcond=None)[0]
#         print(self.coef)
    # y= mx+c
    def query(self,X_test):
        return (self.coef[:-1] * X_test).sum(axis = 1) + self.coef[-1]


## Testing 

In [18]:
dt = DTlearner(size=1)

df = pd.read_csv('Istanbul.csv')
df2 = df.drop('date',axis=1)



X=df2[:50].drop('EM',axis=1)
y=df2[:50]['EM']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

dt.add_evidence(X_train,y_train)
dt.query(X_test)

array([-0.01188111,  0.01868924,  0.0221041 ,  0.0067645 ,  0.0221041 ,
       -0.03832277,  0.0067645 ,  0.0063445 , -0.02342873, -0.00867184,
        0.0067645 , -0.00509224,  0.01281572,  0.00541947,  0.00541947,
        0.01516559,  0.01776444,  0.01868924, -0.02342873,  0.0221041 ])

In [19]:
rt = RTlearner(size=1)

df = pd.read_csv('Istanbul.csv')
df2 = df.drop('date',axis=1)



X=df2[:50].drop('EM',axis=1)
y=df2[:50]['EM']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

rt.add_evidence(X_train,y_train)
rt.query(X_test)

array([-0.02342873,  0.00609936, -0.02001541, -0.03832277, -0.01477275,
       -0.03832277, -0.01909138,  0.00609936,  0.02852446, -0.02001541,
       -0.00209765,  0.02852446, -0.02001541, -0.01477275, -0.02342873,
       -0.00209765,  0.0221041 ,  0.00541947,  0.00541947, -0.02729444])

In [10]:
bl = BagLearner(DTlearner)

df = pd.read_csv('Istanbul.csv')
df2 = df.drop('date',axis=1)



X=df2[:50].drop('EM',axis=1)
y=df2[:50]['EM']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

bl.add_evidence(X_train,y_train)
bl.query(X_test)

array([ 0.01281572, -0.02342873, -0.00555836, -0.00685044,  0.0004087 ,
       -0.00482714, -0.02342873,  0.00224254, -0.02001541,  0.00129031,
        0.01031968,  0.01091689,  0.01031968,  0.00224254,  0.0063445 ,
        0.02490995, -0.01942378, -0.00555836,  0.01868924,  0.00541947])

In [23]:
Ilearner = InsaneLearner()


df = pd.read_csv('Istanbul.csv')
df2 = df.drop('date',axis=1)



X=df2[:50].drop('EM',axis=1)
y=df2[:50]['EM']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

Ilearner.add_evidence(X_train,y_train)
Ilearner.query(X_test)

array([ 0.03031151, -0.00334226, -0.00334226,  0.01031968, -0.00334226,
        0.00224254, -0.00867665,  0.00877264, -0.00334226,  0.01091689,
       -0.00209765, -0.02342873,  0.00967787,  0.01516559, -0.00867665,
       -0.00334226, -0.01477275,  0.01091689, -0.02001541,  0.00129031])

In [24]:
dt = LinRegLearner()

df = pd.read_csv('Istanbul.csv')
df2 = df.drop('date',axis=1)



X=df2[:50].drop('EM',axis=1)
y=df2[:50]['EM']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

dt.add_evidence(X_train,y_train)
display(dt.query(X_test))

20   -0.018936
11   -0.023808
38    0.010743
19   -0.010618
28   -0.011267
22    0.020625
23   -0.002609
48   -0.005600
35    0.000415
18   -0.005243
4    -0.002687
14   -0.009445
46    0.022248
7    -0.026924
5    -0.029842
24    0.022438
36   -0.014240
17    0.022668
49    0.012389
45   -0.004135
dtype: float64